In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data generators
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    brightness_range=[0.7, 1.3],
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "SMARTSYSTEMS/training",
    classes=[
        "bebouwdekom",
        "eindebebouwdekom",
        "snelheid50",
        "zone30",
        "zone50"
    ],
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

#print("Class indices:", train_data.class_indices)

test_data = test_gen.flow_from_directory(
    "SMARTSYSTEMS/testing",
    classes=[
        "bebouwdekom",
        "eindebebouwdekom",
        "snelheid50",
        "zone30",
        "zone50"
    ],
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0003),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Train
model.fit(
    train_data,
    validation_data=test_data,
    epochs=20
)

# Opslaan
model.save("traffic_sign_classifier.keras")


Found 243 images belonging to 5 classes.
Found 78 images belonging to 5 classes.
Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4280 - loss: 1.3478 - val_accuracy: 0.7051 - val_loss: 1.0293
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.7407 - loss: 0.7720 - val_accuracy: 0.8333 - val_loss: 0.7170
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8230 - loss: 0.5172 - val_accuracy: 0.8590 - val_loss: 0.5949
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8148 - loss: 0.4575 - val_accuracy: 0.8846 - val_loss: 0.4793
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8724 - loss: 0.4128 - val_accuracy: 0.8974 - val_loss: 0.4295
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8642 - loss: 0.3380 - val_accuracy: 0.8974 - val_loss: 0.4095
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8765 - loss: 0.3272 - val_accuracy: 0.8846 - val_loss: 0.3682
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - a

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

IMG_SIZE = (224, 224)

model = tf.keras.models.load_model("traffic_sign_classifier.keras")

labels = [
    "bebouwdekom",
    "eindebebouwdekom",
    "snelheid50",
    "zone30",
    "zone50",
]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img = cv2.resize(frame, IMG_SIZE)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)
    class_id = np.argmax(pred)
    confidence = pred[0][class_id]

    label = labels[class_id]
    print(f"Predicted class: {label}, Confidence: {confidence*100:.1f}%")

    text = f"{label} ({confidence*100:.1f}%)"
    cv2.putText(frame, text, (20,40),
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0,255,0), 2)

    cv2.imshow("Traffic Sign Recognition", frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted class: bebouwdekom, Confidence: 65.9%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Predicted class: zone30, Confidence: 44.9%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Predicted class: zone30, Confidence: 47.0%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
Predicted class: bebouwdekom, Confidence: 39.4%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Predicted class: zone30, Confidence: 52.9%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicted class: zone30, Confidence: 58.9%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Predicted class: zone30, Confidence: 55.4%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
Predicted class: bebouwdekom, Confidence: 50.9%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Predicted class: zone30, Confidence: 47.4%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Predicted class: zone30, Confidence: 39.7%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Predicted class: zone30, Confidence: 44.5%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
Predicted class: zone30, Confidence: 65.7%
1/1 